# Transfer Learning With Tensorflow Part 3: Scaling up

We've seen the power of transfer learning feature extraction and fine-tuning, now it's time to scale up to all the classes in Food101 (101 total of classes of food)  
see: https://data.vision.ee.ethz.ch/cvl/datasets_extra/food-101/static/bossard_eccv14_food-101.pdf

## What is transfer learning ?

Transfer learning is leveraging a working model's existing architecture and learned patterns for our own problem

- Can leverage an existing neural network architecture prove to work on problem similiar to our own
- Can leverage a working network architecture which has already learned patterns on similiar data to our own (often results in great results with less data)

Surely someone has spent the time crafting the right model for the job..

Example Transfer Learning use cases:

- Computer vision see: https://www.kaggle.com/competitions/imagenet-object-localization-challenge/overview
- Natural Language Proccessing

## What we're goint to cover

- Download and prepare 10% of all Food101 classes (7500 + training images)
- Training a transfer learning feature extration model
- Fine-tuning our feature extraction model (Food vision mini) to beat the original Food paper with only 10% of the data
- Evaluate Food Vision mini'prediction
- Finding the most wrong predictions (on the dataset)
- Making predictions with food Visin mini on our own custom images

In [1]:
# check to see if we're using a GPU
!nvidia-smi

Fri Mar 15 20:21:56 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.76                 Driver Version: 551.76         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650 Ti   WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   65C    P0             18W /   50W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
import os

In [7]:
STORAGE = os.path.join('..', 'storage')
ZIP_PATH = f'{STORAGE}/zip'
TRANSFER_LEARNING_PATH = f'{STORAGE}/transfer_learning'

In [ ]:
# import urllib.request as ur
# uncomment this line under and run it to download helper_functions file
# ur.urlretrieve('https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py', filename='helper_functions.py')

In [3]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, walk_through_dir

In [4]:
def unzip_data(url, filename, zip_path, transfer_path):
    """
    Unzips filename into the current working directory.

    Args:
        url (str): link from internet where file is located.
        filename (str): a filepath to a target zip folder to be unzipped.
        zip_path (str): path where zip file is located in local machine.
        transfer_zip (str): path where file will be unpackaged from zip.
    """
    import os
    import zipfile
    import urllib.request as ur
    import shutil

    if not os.path.exists(f'{zip_path}/{filename}'):
        # download zip file
        ur.urlretrieve(url, filename)
        shutil.move(filename, f'{zip_path}')

    # unzip the download file
    zip_ref = zipfile.ZipFile(f'{zip_path}/{filename}', 'r')

    # remove folder if exist
    folder = filename.split('.')[0]
    if os.path.isdir(f'{transfer_path}/{folder}'):
        shutil.rmtree(f'{transfer_path}/{folder}')

    zip_ref.extractall(f'{transfer_path}')
    zip_ref.close()

In [12]:
# download files
filename = '101_food_classes_10_percent.zip'
folder = filename.split('.')[0]
url = f'https://storage.googleapis.com/ztm_tf_course/food_vision/{filename}'

unzip_data(url, filename, ZIP_PATH, TRANSFER_LEARNING_PATH)

In [13]:
train_dir = f'{TRANSFER_LEARNING_PATH}/{folder}/train'
test_dir = f'{TRANSFER_LEARNING_PATH}/{folder}/test'

In [14]:
walk_through_dir(f'{TRANSFER_LEARNING_PATH}')

There are 7 directories and 0 images in '..\storage/transfer_learning'.
There are 2 directories and 0 images in '..\storage/transfer_learning\101_food_classes_10_percent'.
There are 101 directories and 0 images in '..\storage/transfer_learning\101_food_classes_10_percent\test'.
There are 0 directories and 250 images in '..\storage/transfer_learning\101_food_classes_10_percent\test\apple_pie'.
There are 0 directories and 250 images in '..\storage/transfer_learning\101_food_classes_10_percent\test\baby_back_ribs'.
There are 0 directories and 250 images in '..\storage/transfer_learning\101_food_classes_10_percent\test\baklava'.
There are 0 directories and 250 images in '..\storage/transfer_learning\101_food_classes_10_percent\test\beef_carpaccio'.
There are 0 directories and 250 images in '..\storage/transfer_learning\101_food_classes_10_percent\test\beef_tartare'.
There are 0 directories and 250 images in '..\storage/transfer_learning\101_food_classes_10_percent\test\beet_salad'.
There a

In [15]:
# setup data inputs
import tensorflow as tf

IMG_SIZE = (224, 224)
train_data_all_10_percent = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                                label_mode='categorical', # 10 classes
                                                                                image_size=IMG_SIZE)
test_data = tf.keras.preprocessing.image_dataset_from_directory(directory=test_dir,
                                                                label_mode='categorical',
                                                                image_size=IMG_SIZE,
                                                                shuffle=False) # don't shuffle test data for prediction analysis

Found 7575 files belonging to 101 classes.
Found 25250 files belonging to 101 classes.
